In [7]:
import pandas as pd

import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from feature_builder import process_dataset, _add_text_embeddings


In [8]:
train_dataset = pd.read_csv('train.csv')

In [9]:
test_dataset = pd.read_csv('test.csv')

In [4]:
y = train_dataset.loc[:,'target']

In [ ]:
clf_processed = svm.SVC()

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x_tfidf = v.fit_transform(train_dataset['text'])


In [24]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf, y, test_size = .33, random_state = 17)

In [25]:
clf_tfidf = svm.SVC()
clf_tfidf.fit(x_train_tfidf, y_train_tfidf)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [84]:
y_pred_tfidf = clf_tfidf.predict(x_test_tfidf)
f1_score(y_test_tfidf, y_pred_tfidf)

0.7261774370208106

Ahora voy a probar con nuestro generador de features que es un conjunto de embeddings con manualidades.

In [10]:
x_processed = process_dataset(train_dataset)

Percentage of words covered in the embeddings = 0.6336399642263958


In [11]:
x_train_processed, x_test_processed, y_train_processed, y_test_processed = train_test_split(x_processed, y, test_size = .33, random_state = 17)

In [12]:
clf_processed.fit(x_train_processed, y_train_processed)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
y_pred_processed = clf_processed.predict(x_test_processed)
f1_score(y_test_processed, y_pred_processed)

0.5270758122743683

Como no son muy buenos resultados voy a probar solo con los embeddings (las primeras 300 columnas)

In [14]:
x_embedd = train_dataset.copy()
_add_text_embeddings(x_embedd)
x_embedd.drop(['text', 'location', 'keyword', 'id', 'target'], axis=1, inplace=True)

Percentage of words covered in the embeddings = 0.6336399642263958


Me quede solo con los embeddings ya que el resto deteriora el algoritmo por alguna razón.

In [15]:
x_train_embedd, x_test_embedd, y_train_embedd, y_test_embedd = train_test_split(x_embedd, y, test_size = .33, random_state = 17)

In [16]:
clf_embeddings = svm.SVC()
clf_embeddings.fit(x_train_embedd, y_train_embedd)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
y_pred_embedd = clf_embeddings.predict(x_test_embedd)
f1_score(y_test_embedd, y_pred_embedd)

0.7602397602397603

Acá obtuve un resultado mucho mejor, por ende lo voy a entrenar de esta manera para todo el dataset y exportarlo a csv.

In [65]:
x_train = train_dataset.copy()
_add_text_embeddings(x_train)
x_train.drop(['text', 'location', 'keyword', 'id', 'target'], axis=1, inplace=True)

In [66]:
x_test = test_dataset.copy()
_add_text_embeddings(x_test)
x_test.drop(['text', 'location', 'keyword', 'id'], axis=1, inplace=True)

In [67]:
clf_embeddings_final = svm.SVC()
clf_embeddings_final.fit(x_train, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [68]:
y_pred_embedd = clf_embeddings_final.predict(x_test)

In [69]:
ids = test_dataset['id']
final_df = pd.DataFrame({'target': [x for x in y_pred_embedd]}, index=ids)

In [71]:
final_df.to_csv('SVM-algo4.csv')